Maxtix factorization can find the relation between users and products iwthout explicit relationship info.

Matrix factorization in recommendation systems is generally considered a collaborative filtering technique, not a content-based filtering approach. Here are the key points to understand this:

1. Collaborative filtering:

- Relies on past user behavior and interactions
- Doesn't require explicit content information about items
- Finds patterns in user-item interactions
2. Matrix factorization as collaborative filtering:
- Uses a user-item interaction matrix
- Decomposes this matrix into lower-dimensional user and item latent factor matrices
- These latent factors capture underlying patterns in user preferences and item characteristics
- Doesn't rely on explicit item features or content
3. How it works:
- Starts with a sparse user-item interaction matrix
- Factorizes this matrix into two lower-dimensional matrices (user factors and item factors)
- These factors represent latent features that explain user preferences and item attributes 
- Predictions are made by multiplying these factor matrices
4. Advantages:
- Can capture complex patterns in user behavior
- Doesn't require content information about items
- Can handle large, sparse datasets efficiently
5. Contrast with content-based filtering:
- Content-based filtering uses explicit features of items (e.g., genre, actors for movies)
- Matrix factorization doesn't require this explicit content information
6. Hybrid approaches:
- Some advanced techniques combine matrix factorization with content-based features for improved performance

- Utilizes Matrix Factorization and KMeans for movie predictions
- Dataset is Movie Lens: # F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <a href="https://doi.org/10.1145/2827872">Link to Paper</a>
-  Associated YouTube Video: <a href="https://youtu.be/G4MBc40rQ2k">Supplemental Material</a>

In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. 

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1133k      0 --:--:-- --:--:-- --:--:-- 1131k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [1]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [2]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)


The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#### Movie ID to movie name mapping

In [22]:
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = ratings_df.userId.nunique()
n_items = ratings_df.movieId.nunique()
print("# users: ", n_users, "\n# items: ", n_items, "\nThe full rating matrix will have: ", n_users*n_items, ' elements.')
print(f"# ratings: {len(ratings_df)}\nTherefore: {len(ratings_df)/(n_users*n_items)*100} % of the matrix is filled.")

# users:  610 
# items:  9724 
The full rating matrix will have:  5931640  elements.
# ratings: 100836
Therefore: 1.6999683055613624 % of the matrix is filled.


We have an <b><i>incredibly sparse matrix </i></b> to work with here.

And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2

You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.

One <b>advantage</b> here is that <u>matrix factorization</u> can realize the rating matrix implicitly, thus we don't need all the data

In [6]:
import torch
import numpy as np
from torch.autograd import Variable

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create embeddings for users
        self.user_factor = torch.nn.Embedding(n_users, n_factors) # it's like a look up table for the input
        # Embedding for items
        self.item_factor = torch.nn.Embedding(n_items, n_factors)
        self.user_factor.weight.data.uniform_(0,0.05)
        self.item_factor.weight.data.uniform_(0,0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factor(users)*self.item_factor(items)).sum(1)
    
    def forward2(self, user, item):
        # matrix multiplication
        return self.user_factor(user)*self.item_factor(item).sum(1)
    
    def predict(self,user, item):
        return self.forward(user, item)

### Creating the dataloader in pytorch

In [7]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader 

class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

### Preparing the training parameters and tools

In [8]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factor): Embedding(610, 8)
  (item_factor): Embedding(9724, 8)
)


/home/mmoha014/.conda/envs/tf-torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from tqdm import tqdm
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))
     

  1%|          | 1/128 [00:03<08:03,  3.81s/it]

iter #0 Loss: 11.063573892951617


  2%|▏         | 2/128 [00:05<05:40,  2.70s/it]

iter #1 Loss: 4.747916829162443


  2%|▏         | 3/128 [00:07<04:52,  2.34s/it]

...

iter #125 Loss: 0.32959684479962753


 99%|█████████▉| 127/128 [04:31<00:02,  2.14s/it]

iter #126 Loss: 0.32910189817323904


100%|██████████| 128/128 [04:33<00:00,  2.14s/it]

iter #127 Loss: 0.3287796936361923


### Fit the clusters based on the movie weights

In [16]:
trained_movie_embeddings = model.item_factor.weight.data.cpu().numpy()

In [17]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [18]:
trained_movie_embeddings

array([[0.7210944 , 0.4119839 , 0.85131705, ..., 0.44539163, 0.32182238,
        0.33362895],
       [0.3040543 , 0.7816727 , 0.59153974, ..., 0.00246055, 0.29792678,
        0.5620495 ],
       [0.70916766, 0.34539557, 0.55498797, ..., 0.38553005, 0.44276807,
        0.240126  ],
       ...,
       [0.3520334 , 0.35725495, 0.35788986, ..., 0.3598867 , 0.376347  ,
        0.35604864],
       [0.4135562 , 0.41313234, 0.41165113, ..., 0.40019327, 0.43589655,
        0.3934105 ],
       [0.39046398, 0.3834934 , 0.41870788, ..., 0.38096786, 0.40370354,
        0.43732196]], dtype=float32)

In [19]:
# !pip install threadpoolctl==3.1.0
import sklearn
sklearn.show_versions()


System:
    python: 3.8.13 (default, Mar 28 2022, 11:38:47)  [GCC 7.5.0]
executable: /home/mmoha014/.conda/envs/tf-torch/bin/python
   machine: Linux-4.18.0-513.9.1.el8_9.x86_64-x86_64-with-glibc2.17

Python dependencies:
      sklearn: 1.1.2
          pip: 22.2.2
   setuptools: 63.4.1
        numpy: 1.19.5
        scipy: 1.9.1
       Cython: None
       pandas: 1.5.0
   matplotlib: 3.7.1
       joblib: 1.2.0
threadpoolctl: 3.1.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: mkl
         prefix: libmkl_rt
       filepath: /home/mmoha014/.conda/envs/tf-torch/lib/libmkl_rt.so.1
        version: 2021.4-Product
threading_layer: intel
    num_threads: 3

       user_api: openmp
   internal_api: openmp
         prefix: libiomp
       filepath: /home/mmoha014/.conda/envs/tf-torch/lib/libiomp5.so
        version: None
    num_threads: 3

       user_api: openmp
   internal_api: openmp
         prefix: libgomp
       filepath: /home/mmoha014/.conda/envs/t

In [20]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)#, random_state=0)
if trained_movie_embeddings is not None and len(trained_movie_embeddings)>0:
    kmeans.fit(trained_movie_embeddings)

It can be seen here that the movies that are in the same cluster tend to have similar genres. 

Also note that the algorithm is unfamiliar with the movie name and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.

In [23]:
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Batman & Robin (1997)
	 Super Mario Bros. (1993)
	 Joe Dirt (2001)
	 Speed 2: Cruise Control (1997)
	 Rocky V (1990)
	 Superman IV: The Quest for Peace (1987)
	 Nutty Professor II: The Klumps (2000)
	 Karate Kid, Part III, The (1989)
	 Shark Tale (2004)
	 Dungeons & Dragons (2000)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Fight Club (1999)
	 Seven (a.k.a. Se7en) (1995)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #2
	 Schindler's List (1993)
	 Toy Story (1995)
	 Aladdin (1992)
	 Back to the Future (1985)
	 Mask, The (1994)
	 Beauty and the Beast (1991)
	 Princess Bride, The (1987)
	 Beautiful Mind, A (2001)
	 E.T. the Extra-Terrestrial (1982)
	 Willy Wonka & the Chocolate Factory (1971)
Cluster #3
	 Iron Man (2008)
	 Starship Trooper